In [1]:
import glob
import numpy as np
import pandas as pd
import csv
import h5py
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Dense, Dropout, LSTM, GRU, Conv1D, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import concatenate, GlobalMaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Reshape, TimeDistributed, Embedding, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax, Adadelta, Adagrad, Nadam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2
from keras import backend as K
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


from pycparser import parse_file as p
#from features import S0Gbg, S1Gbg, S2Gbg, S3Gbg, S4Gbg, S5Gbg, S6Gbg

from numpy.random import seed
from tensorflow.random import set_seed

from keras import backend as K

In [2]:
np.random.seed(0)
Normalise=0.005
tf.random.set_seed(0)

In [3]:
train = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\HMM\MatMul_train_HMM_actual.csv')

In [4]:
#train = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\PSSM\PsePSSM_train_actual.csv')

In [5]:
train

,ACC,Kingdom,Partition,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,Q28165,Metazoa,4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.111411,0.244958,0.341650,0.217299,0.802411,0.559058,0.227327,0.288010,0.182961,1.723475
1,Q86U42,Metazoa,4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.107259,0.262705,0.388817,0.218722,0.858188,0.587983,0.234405,0.282551,0.177832,1.880172
2,Q0GA42,Metazoa,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.356197,0.633754,0.568381,0.466991,0.937301,1.178858,0.828762,1.238542,0.392853,4.291845
3,P82349,Metazoa,1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.093193,0.184774,0.104548,0.178899,0.522350,0.314332,0.218914,0.270916,0.146021,1.486413
4,Q7L5N1,Metazoa,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.171199,0.313611,0.215853,0.317681,0.326752,0.463905,0.392336,0.531924,0.152525,1.729775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,Q09712,Fungi,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.436289,0.538304,0.690612,0.482105,0.727632,1.241338,0.729940,1.201819,0.904024,7.170935
28299,Q9V3C5,Metazoa,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.754753,1.177238,1.066532,1.163425,1.529423,2.532042,1.670486,2.313325,1.660387,5.453517
28300,Q9SQQ9,Viridiplantae,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.127730,0.246381,0.182832,0.391400,0.284849,0.468451,0.171616,0.176120,0.036062,2.257948
28301,Q9SQQ9-3,Viridiplantae,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.125140,0.245346,0.193222,0.421375,0.285701,0.468638,0.185587,0.182496,0.035112,2.267903


In [6]:
X_train = train.drop(['ACC','Kingdom','Partition','Membrane','Cytoplasm','Nucleus','Extracellular','Cell membrane','Mitochondrion','Plastid','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome','Sequence'],axis=1)

In [7]:
X_train

,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Feat_6,Feat_7,Feat_8,Feat_9,Feat_10,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,14.982983,0.201766,1.303502,2.162008,0.533998,2.429846,0.343176,0.647653,1.350559,1.218167,...,0.111411,0.244958,0.341650,0.217299,0.802411,0.559058,0.227327,0.288010,0.182961,1.723475
1,14.216900,0.219421,1.194918,1.976242,0.550106,1.840085,0.336911,0.615057,1.287122,1.204021,...,0.107259,0.262705,0.388817,0.218722,0.858188,0.587983,0.234405,0.282551,0.177832,1.880172
2,15.555804,1.129827,3.171903,3.414553,2.083373,4.350188,1.036959,3.696123,2.462769,5.452501,...,0.356197,0.633754,0.568381,0.466991,0.937301,1.178858,0.828762,1.238542,0.392853,4.291845
3,9.442437,0.327668,0.655006,0.694246,0.384123,1.355365,0.274964,1.001960,0.726103,1.189358,...,0.093193,0.184774,0.104548,0.178899,0.522350,0.314332,0.218914,0.270916,0.146021,1.486413
4,8.554028,0.341592,1.241446,1.367293,0.692368,1.143770,0.502694,1.167590,1.023161,1.898444,...,0.171199,0.313611,0.215853,0.317681,0.326752,0.463905,0.392336,0.531924,0.152525,1.729775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,5.481292,0.715920,0.618349,0.751800,2.638740,2.770277,0.530364,2.928688,0.977403,5.196495,...,0.436289,0.538304,0.690612,0.482105,0.727632,1.241338,0.729940,1.201819,0.904024,7.170935
28299,17.663655,2.106784,4.553804,4.790730,3.127160,6.314527,2.357921,4.745681,3.825695,8.191178,...,0.754753,1.177238,1.066532,1.163425,1.529423,2.532042,1.670486,2.313325,1.660387,5.453517
28300,4.889802,0.275934,0.829372,1.273311,0.398117,1.444185,0.415867,0.665779,1.122971,1.180808,...,0.127730,0.246381,0.182832,0.391400,0.284849,0.468451,0.171616,0.176120,0.036062,2.257948
28301,4.808752,0.267086,0.828082,1.274217,0.391082,1.431766,0.425447,0.667434,1.094252,1.177602,...,0.125140,0.245346,0.193222,0.421375,0.285701,0.468638,0.185587,0.182496,0.035112,2.267903


In [8]:
Y_train = train[['Membrane','Cytoplasm','Nucleus','Extracellular','Cell membrane','Mitochondrion','Plastid','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome']]

In [9]:
Y_train

,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
28298,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
28299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28300,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28301,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Specify the path to your saved model (.h5 file)
model_path = '77.h5'

# Load the model
model = load_model(model_path,compile=False)

# Print a summary of the loaded model
model.summary()

# Now you can use the loaded model for predictions or further training


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hmm_input (InputLayer)      [(None, 400)]             0         
                                                                 
 dense_4 (Dense)             (None, 400)               160400    
                                                                 
 dense_5 (Dense)             (None, 128)               51328     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 main_output (Dense)         (None, 11)                363       
                                                                 
Total params: 222,427
Trainable params: 222,427
Non-trainab

In [11]:
from ClassLabels1 import label_generator
from multiLabelMetrics import accuracy
pred_scores = model.predict(X_train)
arr_Y = np.array(Y_train)
arr_pred = label_generator(predictions=pred_scores)

acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy(),acc.relaxedAccuracy()

885/885 [==============================] - 5s 4ms/step


(44.03420132141469, 89.94163805314565)

In [12]:
correct = pd.DataFrame(columns=range(0, 11))


In [13]:
len(arr_pred)

28303

In [14]:
arr_pred[0]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:

k=0
for i in range(len(arr_pred)):
    if np.array_equal(arr_pred[i], arr_Y[i]):
      # print(k) 
       
       correct = pd.concat([correct, pd.DataFrame([arr_Y[i]])], ignore_index=True)
       

k

0

In [16]:
correct.to_csv('correct1.csv')

In [17]:

test = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\HMM\MatMul_test_HMM_actual.csv')

In [18]:
test

,sid,Cell membrane,Cytoplasm,Endoplasmic reticulum,Golgi apparatus,Lysosome/Vacuole,Mitochondrion,Nucleus,Peroxisome,Lengths,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,ENSP00000355743,0,0,0,0,0,0,1,0,496,...,0.258011,0.353851,0.435284,0.413877,0.594366,0.595805,0.396866,0.544421,0.424152,6.692077
1,ENSP00000429628,0,0,0,0,0,0,1,0,173,...,0.093635,0.185587,0.206210,0.225075,0.214002,0.339029,0.246747,0.250573,0.089538,0.442268
2,ENSP00000266544,0,0,0,0,0,1,0,0,377,...,0.278993,0.339251,0.438263,0.351402,0.544947,0.618431,0.499604,0.843471,0.315650,1.232763
3,ENSP00000470652,0,0,0,1,0,0,0,0,410,...,0.363690,0.433065,0.379793,0.382677,0.473912,0.860343,0.664466,0.945804,0.395950,0.997238
4,ENSP00000460751,0,0,0,0,0,0,1,0,163,...,0.037747,0.119094,0.092005,0.155926,0.215646,0.201815,0.094544,0.163751,0.268752,2.492428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,ENSP00000379284,0,0,0,0,0,1,0,0,372,...,0.302538,0.348803,0.334693,0.324379,0.641153,0.535000,0.460455,0.666990,0.304205,1.033432
1681,ENSP00000430164,0,1,0,0,0,0,0,0,73,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
1682,ENSP00000414655,0,1,0,0,0,0,0,0,174,...,0.014237,0.062561,0.143737,0.114862,0.048551,0.241509,0.102259,0.037878,0.008699,1.638261
1683,ENSP00000392070,0,0,0,0,0,0,1,0,327,...,0.184005,0.214807,0.337576,0.290199,0.442781,0.461941,0.351031,0.541925,0.135670,1.037373


In [19]:
test.dropna(inplace=True)

In [20]:
Xtest=test.drop(['sid','Cell membrane','Cytoplasm','Endoplasmic reticulum','Golgi apparatus','Lysosome/Vacuole','Mitochondrion','Nucleus','Peroxisome','Lengths','fasta'],axis=1)

In [21]:
Xtest

,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Feat_6,Feat_7,Feat_8,Feat_9,Feat_10,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,10.511840,0.681454,1.467145,2.096745,0.502221,1.417815,0.453600,1.072239,1.948673,1.454342,...,0.258011,0.353851,0.435284,0.413877,0.594366,0.595805,0.396866,0.544421,0.424152,6.692077
1,1.657306,0.311132,0.530636,0.704214,0.324607,0.519876,0.353429,0.413341,0.636022,0.694782,...,0.093635,0.185587,0.206210,0.225075,0.214002,0.339029,0.246747,0.250573,0.089538,0.442268
2,7.251676,0.515472,1.437248,1.764437,1.077316,2.332655,0.671138,1.967690,1.592020,3.348442,...,0.278993,0.339251,0.438263,0.351402,0.544947,0.618431,0.499604,0.843471,0.315650,1.232763
3,6.283611,0.732520,0.979360,1.151415,2.139442,3.365095,0.477635,2.617250,1.094336,4.500879,...,0.363690,0.433065,0.379793,0.382677,0.473912,0.860343,0.664466,0.945804,0.395950,0.997238
4,5.599081,0.242164,0.420853,0.709060,0.202405,0.316280,0.257576,0.420584,0.380964,0.757186,...,0.037747,0.119094,0.092005,0.155926,0.215646,0.201815,0.094544,0.163751,0.268752,2.492428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,3.287648,0.363539,1.261358,1.569617,1.170052,1.563395,0.633403,1.594907,1.355015,2.561954,...,0.302538,0.348803,0.334693,0.324379,0.641153,0.535000,0.460455,0.666990,0.304205,1.033432
1681,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
1682,3.704577,0.121874,0.358170,0.334261,0.048148,0.377475,0.187446,0.215600,0.446949,0.381701,...,0.014237,0.062561,0.143737,0.114862,0.048551,0.241509,0.102259,0.037878,0.008699,1.638261
1683,4.817369,0.435462,1.173849,1.843065,0.896056,1.756875,0.508191,1.135122,1.224170,2.156034,...,0.184005,0.214807,0.337576,0.290199,0.442781,0.461941,0.351031,0.541925,0.135670,1.037373


In [22]:
Ytest=test[['Cytoplasm','Nucleus','Cell membrane','Mitochondrion','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome']]

In [23]:
Ytest

,Cytoplasm,Nucleus,Cell membrane,Mitochondrion,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1680,0,0,0,1,0,0,0,0
1681,1,0,0,0,0,0,0,0
1682,1,0,0,0,0,0,0,0
1683,0,1,0,0,0,0,0,0


In [24]:
pred_scores = model.predict(Xtest)
arr_Y = np.array(Ytest)
arr_pred = label_generator(predictions=pred_scores)
print(arr_Y.shape)
print(arr_pred.shape)

53/53 [==============================] - 0s 4ms/step
(1685, 8)
(1685, 11)


In [25]:
df_arr_Y = pd.DataFrame(arr_Y)
df_arr_pred = pd.DataFrame(arr_pred)

df_arr_pred.drop(df_arr_pred.columns[[0,3,6]], axis = 1, inplace = True)

arr_Y = df_arr_Y.values
arr_pred = df_arr_pred.values

In [26]:
print(arr_Y.shape)
print(arr_pred.shape)

(1685, 8)
(1685, 8)


In [27]:
acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

26.468842729970326

In [28]:
acc.relaxedAccuracy()

84.02818991097922

In [29]:
correct = pd.DataFrame(columns=range(0, 11))
k=0
for i in range(len(arr_pred)):
    if np.array_equal(arr_pred[i], arr_Y[i]):
      # print(k) 
       
       correct = pd.concat([correct, pd.DataFrame([arr_Y[i]])], ignore_index=True)
       

In [ ]:
correct.to_csv('correct2.csv')